In [ ]:
from chi import server, context
import chi, os, time, datetime

context.version = "1.0"
context.choose_project()
context.choose_site(default="KVM@TACC")

In [ ]:
username = os.getenv('USER')  # all exp resources will have this prefix
s = server.Server(
    f"node-persist_project_50",
    image_name="CC-Ubuntu24.04",
    flavor_name="m1.large"
)
s.submit(idempotent=True)

In [3]:
s.associate_floating_ip()

In [ ]:
s.refresh()
s.show(type="widget")

In [ ]:
security_groups = [
    {'name': "allow-ssh", 'port': 22, 'description': "Enable SSH traffic on TCP port 22"},
    {'name': "allow-8888", 'port': 8888, 'description': "Enable TCP port 8888 (used for Jupyter)"},
    {'name': "allow-8000", 'port': 8000, 'description': "Enable TCP port 8000 (used for MLFlow)"},
    {'name': "allow-9000", 'port': 9000, 'description': "Enable TCP port 9000 (used for MinIO)"},
    {'name': "allow-9001", 'port': 9001, 'description': "Enable TCP port 9001 (used for MinIO Console)"}
]

# configure openstacksdk for actions unsupported by python-chi
os_conn = chi.clients.connection()
nova_server = chi.nova().servers.get(s.id)
for sg in security_groups:
    if not os_conn.get_security_group(sg['name']):
        os_conn.create_security_group(sg['name'], sg['description'])
        os_conn.create_security_group_rule(sg['name'], port_range_min=sg['port'], port_range_max=sg['port'], protocol='tcp')
    nova_server.add_security_group(sg['name'])
print(f"updated security groups: {[group.name for group in nova_server.list_security_group()]}")
s.refresh()
s.check_connectivity()

In [ ]:
s.execute("git clone https://github.com/DeepakSingh260/Real-Time-News-Recommnedation-System")
s.execute("curl -sSL https://get.docker.com/ | sudo sh")
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER")

In [ ]:
context.choose_site(default="CHI@TACC")

# Create object storage container
object_container_name = f"object-persist-project-50"
os_conn = chi.clients.connection()
os_conn.create_container(object_container_name)
print(f"Created object container: {object_container_name}")


In [ ]:
context.choose_site(default="KVM@TACC")

# Create a block storage volume
cinder_client = chi.clients.cinder()
volume_name = f"block-persist-project-50"
volume = cinder_client.volumes.create(name=volume_name, size=5)  # 5 GiB
print(f"Created block storage volume: {volume_name}")

In [ ]:
volume_manager = chi.nova().volumes
volume_manager.create_server_volume(server_id=s.id, volume_id=volume.id)
print(f"Attached volume {volume_name} to server node-persist_project_50")

In [ ]:
rclone_setup = """
curl https://rclone.org/install.sh | sudo bash
sudo sed -i '/^#user_allow_other/s/^#//' /etc/fuse.conf
mkdir -p ~/.config/rclone
cat > ~/.config/rclone/rclone.conf << EOF
[chi_tacc]
type = swift
user_id = your_user_id
application_credential_id = your_application_credential_id
application_credential_secret = your_application_credential_secret
auth = https://chi.tacc.chameleoncloud.org:5000/v3
region = CHI@TACC
EOF
"""

s.execute(rclone_setup)

In [ ]:
format_mount_cmd = """
sudo parted -s /dev/vdb mklabel gpt
sudo parted -s /dev/vdb mkpart primary ext4 0% 100%
sudo mkfs.ext4 /dev/vdb1
sudo mkdir -p /mnt/block
sudo mount /dev/vdb1 /mnt/block
sudo chown -R cc /mnt/block
sudo chgrp -R cc /mnt/block
"""

s.execute(format_mount_cmd)

In [ ]:
# Create a volume for the ETL pipeline
docker_volume_cmd = "docker volume create mind"
s.execute(docker_volume_cmd)

# Run the extract data stage
extract_cmd = "docker compose -f ~/Real-Time-News-Recommnedation-System/Data\\ Pipeline/object_storage/docker-compose-mind-etl.yaml run download-data"
s.execute(extract_cmd)

In [ ]:
transform_cmd = "docker compose -f ~/Real-Time-News-Recommnedation-System/Data\\ Pipeline/object_storage/docker-compose-mind-etl.yaml run transform-data"
s.execute(transform_cmd)

In [ ]:
load_cmd = f"RCLONE_CONTAINER={object_container_name} docker compose -f ~/Real-Time-News-Recommnedation-System/Data\\ Pipeline/object_storage/docker-compose-mind-etl.yaml run load-data"
s.execute(load_cmd)


In [ ]:
mount_cmd = f"""
sudo mkdir -p /mnt/object
sudo chown -R cc /mnt/object
sudo chgrp -R cc /mnt/object
rclone mount chi_tacc:{object_container_name} /mnt/object --read-only --allow-other --daemon
"""
s.execute(mount_cmd)

In [ ]:
# After mounting object storage, set up MLFlow and other services
get_ip_cmd = "hostname -I | awk '{print $1}'"
host_ip_result = s.execute(get_ip_cmd)
host_ip = host_ip_result.stdout.strip()
print(f"Host IP: {host_ip}")

# Get the floating IP for external access
get_floating_ip_cmd = "curl -s ifconfig.me"
floating_ip_result = s.execute(get_floating_ip_cmd)
floating_ip = floating_ip_result.stdout.strip()
print(f"Floating IP: {floating_ip}")

# Create directories on block storage for services
dirs_cmd = """
mkdir -p /mnt/block/postgres_data
mkdir -p /mnt/block/minio_data
chmod 775 /mnt/block/postgres_data
chmod 775 /mnt/block/minio_data
"""
s.execute(dirs_cmd)

# Start the services using Docker Compose
start_cmd = f"""
cd ~/Real-Time-News-Recommnedation-System/Data\\ Pipeline/block_storage/ && 
HOST_IP={floating_ip} docker compose -f docker-compose-mind-complete.yaml up -d
"""
start_result = s.execute(start_cmd)
print(f"Started services: {start_result.stdout}")

# Create a Docker network for services to communicate
network_cmd = "docker network create news_network || true"
s.execute(network_cmd)
print("Created Docker network")

# Connect existing containers to the network
connect_cmd = """
for container in $(docker ps --format '{{.Names}}'); do
    docker network connect news_network $container || true
done
"""
s.execute(connect_cmd)
print("Connected containers to network")

# Start Jupyter with access to our data and services
jupyter_cmd = f"""
docker run -d --rm \\
    -p 8888:8888 \\
    --shm-size 8G \\
    --network news_network \\
    -e MLFLOW_TRACKING_URI=http://{floating_ip}:8000 \\
    -v ~/Real-Time-News-Recommnedation-System:/home/jovyan/work/ \\
    --mount type=bind,source=/mnt/object,target=/mnt/news_data,readonly \\
    --name jupyter_news \\
    quay.io/jupyter/pytorch-notebook:latest
"""
jupyter_result = s.execute(jupyter_cmd)
print(f"Started Jupyter: {jupyter_result.stdout}")

# Install required packages in Jupyter
packages_cmd = "docker exec jupyter_news pip install mlflow torch requests pandas numpy matplotlib scikit-learn"
s.execute(packages_cmd)
print("Installed packages in Jupyter")